In [1]:
"""Bayesian way"""
import arviz as az
import stan
import nest_asyncio
nest_asyncio.apply()

"""Basic ones"""
import polars as pl
from plotnine import *
import matplotlib.pyplot as plt
plt.show()  # for display figures after ggplot
# plt.rcParams['axes.grid'] = True
import numpy as np
import seaborn as sns
import pandas
import statsmodels.api as sm
from datetime import datetime

import warnings
warnings.simplefilter("ignore")
import gc

In [2]:
df_hrv_freq = pl.read_csv("../../data/HRV/freqency_domain.csv").with_columns((pl.col("LF") / pl.col("HF")).alias("LFHF"))
df_hrv_timenon = pl.read_csv("../../data/HRV/time_nonlinear_domain.csv").drop("mean_RRI")
df_hrv = df_hrv_freq.join(df_hrv_timenon, on="date")
df_hrv = df_hrv.with_columns(pl.col("date").str.strptime(pl.Date, "%Y/%m/%d"))
del df_hrv_freq, df_hrv_timenon
gc.collect()
df_hrv

date,VLF,LF,HF,LFHF,HR,SDNN,RMSSD,pNN50,TINN,CD,ApEn,SD1,SD2,SD1SD2
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2017-07-24,884.554467,176.079683,98.624315,1.785358,89.918794,50.293799,20.861884,0.024854,226.5625,0.208329,0.922786,14.75156,69.499062,0.212256
2017-07-25,977.6705,271.4607,114.9889,2.360756,88.66,57.4209,50.0331,0.0659,140.625,0.2131,0.9482,35.3787,72.7817,0.4861
2017-07-26,833.683813,284.855324,167.546256,1.700159,93.593945,51.727819,27.178771,0.031002,195.3125,0.1454239,0.634698,19.218055,70.521594,0.272513
2017-07-27,1636.069588,293.085583,222.901403,1.314866,83.92394,75.542652,38.155843,0.062075,304.6875,0.16728,0.728771,26.979802,102.905322,0.262181
2017-07-28,994.366727,335.918425,517.969574,0.648529,90.402819,59.249159,41.522311,0.053435,171.875,0.140829,0.58117,29.360692,78.456813,0.374227
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-27,546.676301,131.966282,175.335502,0.75265,92.190444,58.001563,38.307472,0.025624,234.375,0.16321,0.823067,27.087459,77.440066,0.349786
2024-05-28,507.299106,106.736518,61.719957,1.729368,91.105591,55.534079,15.998736,0.006512,234.375,0.148263,0.82495,11.312079,77.598669,0.145777
2024-05-29,959.257511,210.958493,314.956597,0.669802,90.607292,85.148795,41.667903,0.013879,109.375,0.129014,0.4432,29.453103,109.239324,0.26962


In [3]:
df_weather = pl.read_csv("../../data/weather/weather.csv")
df_weather = df_weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"))
weather_columns = df_weather.columns
df_weather

date,day_of_week,mean_press,min_press_sea,mean_press_sea,mean_vapor,mean_hum,min_hum,total_preci,hourly_max_preci,max_depth,total_snowfall,sun_hour,mean_temp,max_temp,min_temp,mean_wind_speed,max_wind_speed,max_gust,most_direction,most_direction_dummy
date,str,f64,f64,f64,f64,i64,i64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,i64
2017-07-01,"""土""",983.8,1007.1,1008.1,23.8,88,63,11.5,4.0,0,0,0.3,22.5,25.1,20.3,1.5,3.8,6.2,"""南南東""",7
2017-07-02,"""日""",985.0,1007.5,1009.3,25.9,91,63,34.0,14.0,0,0,0.7,23.3,29.2,20.3,1.8,5.7,9.6,"""北東""",2
2017-07-03,"""月""",982.8,1004.0,1006.9,26.3,89,56,52.0,10.0,0,0,2.4,24.1,29.3,21.7,2.2,6.3,11.3,"""北""",16
2017-07-04,"""火""",982.3,1004.7,1006.6,25.6,97,87,37.0,11.0,0,0,0.0,22.0,24.2,20.1,1.2,3.6,5.3,"""北""",16
2017-07-05,"""水""",983.7,1004.0,1007.9,23.7,82,62,0.0,0.0,0,0,5.5,23.6,27.8,20.6,2.8,6.7,9.8,"""北西""",14
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-27,"""月""",979.4,999.0,1003.8,17.4,69,41,0.0,0.0,0,0,3.6,21.6,29.9,14.9,1.8,4.9,7.6,"""南""",8
2024-05-28,"""火""",978.1,993.7,1002.8,16.7,89,72,24.5,6.0,0,0,0.0,16.5,20.3,14.5,2.0,6.3,10.8,"""北西""",14
2024-05-29,"""水""",984.8,1000.1,1009.7,14.4,80,64,0.0,2.5,0,0,4.0,15.9,20.2,11.8,4.1,8.9,15.1,"""西""",12


In [4]:
df_hrv.join(df_weather, on="date").null_count().sum(axis=1)

sum
u32
42


In [6]:
df = df_hrv.join(df_weather, on="date")
nulls = df.hstack(df.transpose().select(pl.all().is_null().sum()).transpose().rename({"column_0": "null_count"}))
nulls.filter(pl.col("null_count") != 0)[weather_columns]
# nulls.filter(pl.col("null_count") != 0).select(pl.exclude(weather_columns))

VLF,LF,HF,LFHF,HR,SDNN,RMSSD,pNN50,TINN,CD,ApEn,SD1,SD2,SD1SD2,null_count
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32
658.703687,128.829987,94.158597,1.368223,98.859369,46.419755,20.036842,0.035347,109.375,0.118878,0.479434,14.167586,63.775902,0.222146,3
774.232412,168.688253,154.607429,1.091075,104.818948,59.624161,25.878161,0.061238,156.25,0.170278,0.652095,18.298444,82.279901,0.222393,3
552.670902,98.857746,32.737465,3.019713,84.460946,44.447735,15.093479,0.002851,179.6875,0.196019,0.925189,10.672599,61.865873,0.172512,3
529.911144,59.768626,42.634155,1.401895,103.916035,47.002602,14.376169,0.017787,148.4375,0.127465,0.508416,10.165378,65.537233,0.155108,2
504.284985,62.560337,21.007735,2.977967,101.521875,38.60067,7.431113,0.0,62.5,0.089808,0.635683,5.254025,54.271225,0.096811,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
384.450606,100.960142,106.11015,0.951465,111.5694,37.718423,24.067123,0.064895,70.3125,0.168393,0.687783,17.017887,50.485585,0.337084,2
286.599295,96.207511,39.820731,2.416016,94.446781,50.181837,10.200481,0.001555,226.5625,0.116358,0.57319,7.212621,70.58555,0.102183,5
713.035776,98.110013,34.757787,2.822677,101.805032,61.432178,8.317435,0.002649,109.375,0.083695,0.290618,5.880648,86.613636,0.067895,5
